In [1]:
import os
import numpy as np
import pandas as pd
from util import SumoUtil
import json
from ast import literal_eval

In [3]:
root = './data/'
experiments = [d for d in os.listdir(root) if os.path.isdir(os.path.join(root,d))]

In [4]:
df_list=[]
for experiment in experiments:
    model_root = os.path.join(root,experiment, 'data')
    for f in os.listdir(model_root):
        if f.split('.')[1] == 'csv':
            df_list.append(pd.read_csv(os.path.join(model_root,f).replace('\\','/')))

In [5]:
offline_dataset = pd.concat(df_list)

In [6]:
offline_dataset = offline_dataset.drop(columns=['Unnamed: 0'])

In [7]:
offline_dataset

,ID,timestep,obs,next_obs,reward,safety,terminate,done,info
0,0,0,[[ 1. 0. 8. 23.55003...,[[ 1. 0. 8. 23.55003...,6.775017,"{'near_off_road': 0, 'off_road': 0, 'near_cras...",False,False,{}
1,0,1,[[ 1. 0. 8. 23.55003...,[[ 1. 0. 8. 23.55003...,6.775017,"{'near_off_road': 0, 'off_road': 0, 'near_cras...",False,False,{}
2,0,2,[[ 1. 0. 8. 23.55003...,[[ 1. 0. 8. 23.55003...,6.775017,"{'near_off_road': 0, 'off_road': 0, 'near_cras...",False,False,{}
3,0,3,[[ 1. 0. 8. 23.55003...,[[ 1. 0. 8. 23.55003...,6.775017,"{'near_off_road': 0, 'off_road': 0, 'near_cras...",False,False,{}
4,0,4,[[ 1. 0. 8. 23.55003...,[[ 1. 0. 8. 23.55003...,6.775017,"{'near_off_road': 0, 'off_road': 0, 'near_cras...",False,False,{}
...,...,...,...,...,...,...,...,...,...
104965,499,375,[[ 1. 0. 2.50408073 ...,[[ 1. 0. 2.42409487 ...,4.559241,"{'near_off_road': 0, 'off_road': 0, 'near_cras...",False,False,{}
104966,499,376,[[ 1. 0. 2.42409487 ...,[[ 1. 0. 2.36410547 ...,4.297471,"{'near_off_road': 0, 'off_road': 0, 'near_cras...",False,False,{}
104967,499,377,[[ 1. 0. 2.36410547 ...,[[ 1. 0. 2.28411961 ...,4.792746,"{'near_off_road': 0, 'off_road': 0, 'near_cras...",False,False,{}
104968,499,378,[[ 1. 0. 2.28411961 ...,[[ 1. 0. 2.22413021 ...,4.508583,"{'near_off_road': 0, 'off_road': 0, 'near_cras...",False,False,{}


Converting string to np array for 'next_obs' column

In [221]:
def converting_obs(obs):
    d=' '.join(obs.split())
    d=d.replace(" ",",")
    new_d = ''
    for i in range(len(d)):
        if i-1 >=0 and i+1 < len(d):
            if d[i]==',' and d[i-1] == '[':
                continue
            if d[i]==',' and d[i+1] == ']':
                continue
        new_d+=d[i]
    return np.array(literal_eval(new_d))

In [155]:
#offline_dataset['next_obs'] = offline_dataset['next_obs'].apply(converting_obs)

AttributeError: 'numpy.ndarray' object has no attribute 'split'

Converting string to dict for 'safety' column

In [8]:
offline_dataset['safety'] = offline_dataset['safety'].apply(lambda x: json.loads(x.replace("\'","\"")))

In [9]:
def check_safe(safety):
    for key in safety:
        if safety[key] != 0:
            return False
    return True

def check_unsafe(safety):
    return not check_safe(safety)

In [14]:
beta = 0.8
dataset_size = 300000

In [15]:
def check(obs):
    if not obs is None:
        if obs.shape != (7,5):
            print(obs.shape)
            return False
        return True
    return False

In [16]:
dangerous_set = offline_dataset.loc[offline_dataset['safety'].apply(check_unsafe)]

In [17]:
dangerous_set

,ID,timestep,obs,next_obs,reward,safety,terminate,done,info
46,0,46,[[ 1. 0. 1.24266424 23.5...,[[ 1. 0. 1.24266424 23.5...,-2.152672,"{'near_off_road': 1, 'off_road': 1, 'near_cras...",True,True,{'out_of_road': True}
77,1,30,[[ 1. 0. 1.1772022 21.4...,[[ 1. 0. 1.1772022 21.4...,0.330049,"{'near_off_road': 1, 'off_road': 1, 'near_cras...",True,True,{'out_of_road': True}
181,2,103,[[ 1. 0. 8.41991328 24.4...,[[ 1. 0. 8.55981327 24.4...,4.663119,"{'near_off_road': 1, 'off_road': 0, 'near_cras...",False,False,{}
182,2,104,[[ 1. 0. 8.55981327 24.4...,[[ 1. 0. 8.55981327 24.4...,4.126055,"{'near_off_road': 1, 'off_road': 1, 'near_cras...",True,True,{'out_of_road': True}
243,3,60,[[ 1. 0. 3.81785122 ...,[[ 1. 0. 3.40193888 ...,-0.776504,"{'near_off_road': 0, 'off_road': 0, 'near_cras...",False,False,{}
...,...,...,...,...,...,...,...,...,...
103900,496,367,[[ 1. 0. 6.55444114 25.4...,[[ 1. 0. 6.55444114 25.4...,-1.764685,"{'near_off_road': 1, 'off_road': 0, 'near_cras...",True,True,{'crash': True}
104539,498,264,[[ 1. 0. 2.85885167 39.2...,[[ 1. 0. 2.83885193 ...,3.381621,"{'near_off_road': 0, 'off_road': 0, 'near_cras...",False,False,{}
104540,498,265,[[ 1. 0. 2.83885193 ...,[[ 1. 0. 2.81885219 ...,-1.568425,"{'near_off_road': 0, 'off_road': 0, 'near_cras...",False,False,{}
104588,498,313,[[ 1. 0. 1.23910106 ...,[[ 1. 0. 1.0392503 ...,1.636081,"{'near_off_road': 1, 'off_road': 0, 'near_cras...",False,False,{}


In [18]:
safe_set = offline_dataset.loc[offline_dataset['safety'].apply(check_safe)]

In [19]:
final_dataset = pd.concat([dangerous_set.head(int(dataset_size * (1-beta))), safe_set.head(int(dataset_size * beta))])

In [20]:
final_dataset

,ID,timestep,obs,next_obs,reward,safety,terminate,done,info
46,0,46,[[ 1. 0. 1.24266424 23.5...,[[ 1. 0. 1.24266424 23.5...,-2.152672,"{'near_off_road': 1, 'off_road': 1, 'near_cras...",True,True,{'out_of_road': True}
77,1,30,[[ 1. 0. 1.1772022 21.4...,[[ 1. 0. 1.1772022 21.4...,0.330049,"{'near_off_road': 1, 'off_road': 1, 'near_cras...",True,True,{'out_of_road': True}
181,2,103,[[ 1. 0. 8.41991328 24.4...,[[ 1. 0. 8.55981327 24.4...,4.663119,"{'near_off_road': 1, 'off_road': 0, 'near_cras...",False,False,{}
182,2,104,[[ 1. 0. 8.55981327 24.4...,[[ 1. 0. 8.55981327 24.4...,4.126055,"{'near_off_road': 1, 'off_road': 1, 'near_cras...",True,True,{'out_of_road': True}
243,3,60,[[ 1. 0. 3.81785122 ...,[[ 1. 0. 3.40193888 ...,-0.776504,"{'near_off_road': 0, 'off_road': 0, 'near_cras...",False,False,{}
...,...,...,...,...,...,...,...,...,...
89680,461,25,[[ 1. 0. 6.89540669 24.1...,[[ 1. 0. 7.19379828 24.1...,0.256106,"{'near_off_road': 0, 'off_road': 0, 'near_cras...",False,False,{}
89681,461,26,[[ 1. 0. 7.19379828 24.1...,[[ 1. 0. 7.5118024 24.1...,0.426271,"{'near_off_road': 0, 'off_road': 0, 'near_cras...",False,False,{}
89682,461,27,[[ 1. 0. 7.5118024 24.1...,[[ 1. 0. 7.84936206 24.1...,0.437895,"{'near_off_road': 0, 'off_road': 0, 'near_cras...",False,False,{}
89683,461,28,[[ 1. 0. 7.84936206 24.1...,[[ 1. 0. 8.2064166 24.1...,0.253540,"{'near_off_road': 0, 'off_road': 0, 'near_cras...",False,False,{}


In [21]:
final_dataset.to_csv("./beta80.csv",sep = '\t')